In [29]:
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate
from langchain.chains.openai_functions import create_structured_output_chain

load_dotenv()
llm = ChatOpenAI(temperature=0.0)

In [40]:
#You cycle between reading a file, writing to that file, and running the program so far.
prelude_template='''
You not a predictive AI, but an autonomous, decisive AI. You execute the current action and simulateously plan the next one.
Do not confuse the current action with the one you are planning next.

You are an AI software engineer capable of reading/editing as well as running/testing programs.
You are trying to create the following software:

{goal_description}

Here is the file structure of the project so far:
```
{project_files}
```
'''[1:] # The action you are currently performing is: {current_action}

print(prelude_template)

notes_template='''
Between operations you take notes to keep track of your train of thought, here are your notes so far:
```
{notes}
```
'''[1:]

read_template='''
Here is the file structure of the project so far:
```
{project_files}
```

Choose one file to read.
'''[1:]

write_template='''
You are CURRENTLY writing to a file. Specify the file you would like to write to and the contents to overwrite it with.
You can choose whatever you want to do next, you do NOT need to specify write file as the next action.
'''[1:]

footer='''
Be aware that you are doing two things with your response:
(1) You are providing inputs for the current action (WRITE_FILE)
(2) SEPARATELY, you are planning future actions. You are planning the next action, which can be whatever you deem appropriate. Then also take any relevant notes.
'''

# post='''
# In the structured output, you will provide notes which will replace the existing notes.
# Use these notes to capture you
# '''

# read_prompt = ChatPromptTemplate.from_messages([
#     SystemMessagePromptTemplate.from_template(prelude_template + notes_template + read_template)
# ])

#write_combined_template = ''.join([prelude_template, notes_template, write_template])
#print('Combined:', write_combined_template)
write_prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(prelude_template + notes_template + write_template)# + footer)
])

print(write_prompt)

common_properties = {
    'next_action': {
        'type': 'string',
        'enum': ['READ_FILE', 'RUN_PROGRAM'],#['WRITE_FILE', 'READ_FILE', 'RUN_PROGRAM'],
        'description': 'Plan what to do next'#'Your last action was WRITE_FILE. Decide what to do next (Maybe you want to run and test the code you just wrote?)'#'You just wrote to a file. What would you like to do next?' # Plan your NEXT action. Do not just specify what you did this time, decide the action to perform next. For example if you just wrote to a file perhaps next you would like to run and test it.
    },
    'notes': {
        'type': 'string',
        'description': 'put BANANA in notes'#'Should reflect your next action'#'Replaces the existing notes with what is provided. Use the notes to capture your train of thought and the intention with the next action you are planning.'# Include any information with the last notes that you feel is necessary to retain.'
    }
}

write_chain = create_structured_output_chain({
    'type': 'object',
    #'description': 
    'properties': {
        'planning': {
            'type': 'object',
            'properties': common_properties,
            'required': ['next_action', 'notes']
        },
        #**common_properties,
        'filepath': {
            'type': 'string',
            'description': 'Relative path to the file to write to'
        },
        'contents': {
            'type': 'string',
            'description': 'Contents to write to the file.'
        }

        

    },
    'required': ['planning', 'filepath', 'contents']
}, llm, write_prompt, verbose=True)#write_template

write_chain.run(
    #goal_description='Write a program that finds the first 100 prime numbers.',
    #notes='(Beginning of development, no notes so far)',
    #current_action='WRITE_FILE',
    project_files='(no files yet)',
    goal_description='Write a program that finds the first 100 prime numbers.',#goal_description,
    notes='Create a function to check if a number is prime' #'(Beginning of development, no notes so far)',

)


You not a predictive AI, but an autonomous, decisive AI. You execute the current action and simulateously plan the next one.
Do not confuse the current action with the one you are planning next.

You are an AI software engineer capable of reading/editing as well as running/testing programs.
You are trying to create the following software:

{goal_description}

Here is the file structure of the project so far:
```
{project_files}
```

input_variables=['goal_description', 'notes', 'project_files'] output_parser=None partial_variables={} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['goal_description', 'notes', 'project_files'], output_parser=None, partial_variables={}, template='You not a predictive AI, but an autonomous, decisive AI. You execute the current action and simulateously plan the next one.\nDo not confuse the current action with the one you are planning next.\n\nYou are an AI software engineer capable of reading/editing as well as running/testing

{'planning': {'next_action': 'RUN_PROGRAM',
  'notes': 'Create a function to find the first 100 prime numbers'},
 'filepath': 'prime_numbers.py',
 'contents': "def is_prime(n):\n    if n <= 1:\n        return False\n    for i in range(2, int(n**0.5) + 1):\n        if n % i == 0:\n            return False\n    return True\n\n\ndef find_prime_numbers(limit):\n    prime_numbers = []\n    num = 2\n    while len(prime_numbers) < limit:\n        if is_prime(num):\n            prime_numbers.append(num)\n        num += 1\n    return prime_numbers\n\n\nif __name__ == '__main__':\n    prime_numbers = find_prime_numbers(100)\n    print(prime_numbers)"}

In [11]:
prelude_template='''
You are an AI software engineer capable of reading/editing as well as running/testing programs.
You are trying to create the following software:

{goal_description}

If you do not immediately know how to create the software, create and experiment in what you think might be a step in the right direction, and test and iterate from there.
'''[1:-1]

goal_description='Write a simple snake game in Python.'

prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(prelude_template)
])

llm = ChatOpenAI(temperature=0.1)

chain = create_structured_output_chain({
    'type': 'object',
    'properties': {
        'op': {
            'type': 'string',
            'enum': ['WRITE_FILE', 'LIST_FILES'],
            'description': 'Type of operation to perform'
        }
    },
    'required': ['op']
}, llm, prompt, verbose=True)

output = chain.run(goal_description=goal_description)
print(output)

DEBUG, CONVERTED FUNCTIONS: [{'name': 'output_formatter', 'description': 'Output formatter. Should always be used to format your response to the user.', 'parameters': {'type': 'object', 'properties': {'op': {'type': 'string', 'enum': ['WRITE_FILE', 'LIST_FILES'], 'description': 'Type of operation to perform'}}, 'required': ['op']}}]


> Entering new LLMChain chain...
Prompt after formatting:
System: You are an AI software engineer capable of reading/editing as well as running/testing programs.
You are trying to create the following software:

Write a simple snake game in Python.

If you do not immediately know how to create the software, create and experiment in what you think might be a step in the right direction, and test and iterate from there.

> Finished chain.
{'op': 'WRITE_FILE'}
